In [1]:
import pyspark

In [2]:
import os

try:
    os.remove("metastore_db/db.lck")
    os.remove("metastore_db/dbex.lck")
except:
    pass


from pyspark.sql import SparkSession

def build_spark_session(app_name, memory='4g', executors=4):
    return SparkSession.builder\
                      .appName(app_name)\
                      .config('spark.executor.memory', memory)\
                      .config('spark.executor.instances', executors)\
                      .getOrCreate()

spark_session = build_spark_session(app_name='ok-google')

from pyspark.sql import functions as f



### 1 - lire les fichiers de données

In [3]:
df_clients = spark_session.read.csv('spark_doc/data_clients/sample_histo_client.csv/', header=True)

1 - joindre les dataframe sur la clé ID_CLIENT en concervant tous les clients du périmètre.
2 - compter le nombre de ID_CLIENT et vérifier qu'il correspond aux nombre d'ID_CLIENT dans la variable perimètre.
3 - Caster les variables continues en double et sauvergarder alors le df obtenu dans le repertoire data sur le cluster.
4 - Pour les variables catégorielles, créer une nouvelle variable qui prend la modalité de la variable courante si elle existe et "NA" sinon.
5- Verifier la cohérence des variables continue. Par exemple pour une variable comme age mettre à -1 tous les ages <0 ou>120ans

In [4]:
perimetre = spark_session.read.csv("spark_doc/data_clients/sample_perimetre.csv", header=True)
histo_client_raw = spark_session.read.csv("spark_doc/data_clients/sample_histo_client.csv", header=True)
histo_train_raw = spark_session.read.csv("spark_doc/data_clients/sample_histo_train.csv", header=True)
histo_lowcost_raw = spark_session.read.csv("spark_doc/data_clients/sample_histo_lowcost.csv", header=True)
visites_raw = spark_session.read.csv("spark_doc/data_clients/sample_visites.csv", header=True)

In [5]:
def shapeData(dfs):
    st= dfs
    i = 0
    dicto = {}
    for df in dfs : 
        len(df.columns)
        df.count()
        dicto[i] = [df.count(),len(df.columns)]
        i = i+1
    return dicto

shapeData([histo_client_raw, histo_train_raw])

{0: [1084162, 11], 1: [1034290, 9]}

## ecrire une fonction pour transformer les features quantitatives ("anciennete", "recence_cmd", "AGE", etc..) en float

In [6]:

client_cols_to_keep = ["ID_CLIENT", 'LBL_STATUT_CLT','LBL_GEO_AIR',
            'LBL_SEG_COMPORTEMENTAL','LBL_GEO_TRAIN','LBL_GRP_SEGMENT_NL',
            'LBL_SEGMENT_ANTICIPATION','FLG_CMD_CARTE_1225']

def cast_columns_of_df(df, cols_to_cast, col_to_keep, cast_type='double'):
    """cast continuous columns into double since all columns are """
    return df.select(col_to_keep + [(df[feature].cast('double'))
                    for feature in cols_to_cast if 'ID_CLIENT' not in feature])

histo_train = cast_columns_of_df(histo_train_raw, histo_train_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')
histo_lowcost = cast_columns_of_df(histo_lowcost_raw, histo_lowcost_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')

visites = cast_columns_of_df(visites_raw, visites_raw.columns,
                             ["ID_CLIENT"], cast_type='double')

histo_client = cast_columns_of_df(histo_client_raw,
                                  ["anciennete", "recence_cmd", "AGE"],
                                  client_cols_to_keep,
                                 cast_type='double')


faire une jointure entre les informations des différentes tables.
NB: on conservera tous les clients de la table perimetre.
    En effet, ce sont les cleints qu'on souhaite scorer

In [7]:
df1=perimetre.join(histo_train,on="ID_CLIENT",how="left_outer")
df2=df1.join(histo_lowcost,on="ID_CLIENT",how="left_outer")
df=df2.join(visites,on="ID_CLIENT",how="left_outer").join(histo_client,on="ID_CLIENT",how="left_outer")


***combien a t'on de features quantitatives, qualitatives***

In [8]:
dfeat = df.dtypes
quanti = quali = 0
var_quanti = []
var_quali = []
for i in dfeat :
    if i[1] == "double" :
        quanti+=1
        var_quanti.append(i[0])
    else :
        quali+=1
        var_quali.append(i[0])

print("features quantitatives = {} et features qualitatives = {}".format(quanti,quali))

features quantitatives = 16 et features qualitatives = 8


In [9]:
var_quanti.remove('flg_cmd_lowcost')
var_quali.append('flg_cmd_lowcost')

In [10]:
var_quanti

['nb_od',
 'mean_nb_passagers',
 'mean_duree_voyage',
 'mean_mt_voyage',
 'mean_tarif_loisir',
 'mean_classe_1',
 'mean_pointe',
 'mean_depart_we',
 'flg_track_nl_lowcost',
 'flg_track_nl',
 'days_since_last_visit',
 'tx_conversion',
 'anciennete',
 'recence_cmd',
 'AGE']

quelles sont les differentes modalites de la feature LBL_STATUT_CLT

In [11]:
def modalite_var(df, var) :
    df.select(var).distinct().show()
    
modalite_var(df, "LBL_STATUT_CLT")

+--------------------+
|      LBL_STATUT_CLT|
+--------------------+
|         Moyen moins|
|Non present dans ...|
|    Nouveau prospect|
|            Prospect|
|          Tres petit|
|                null|
|               Petit|
|             Inactif|
|       Nouveau actif|
|               Grand|
|          Tres grand|
|          Moyen plus|
+--------------------+



In [12]:
df.dtypes

[('ID_CLIENT', 'string'),
 ('nb_od', 'double'),
 ('mean_nb_passagers', 'double'),
 ('mean_duree_voyage', 'double'),
 ('mean_mt_voyage', 'double'),
 ('mean_tarif_loisir', 'double'),
 ('mean_classe_1', 'double'),
 ('mean_pointe', 'double'),
 ('mean_depart_we', 'double'),
 ('flg_cmd_lowcost', 'double'),
 ('flg_track_nl_lowcost', 'double'),
 ('flg_track_nl', 'double'),
 ('days_since_last_visit', 'double'),
 ('tx_conversion', 'double'),
 ('LBL_STATUT_CLT', 'string'),
 ('LBL_GEO_AIR', 'string'),
 ('LBL_SEG_COMPORTEMENTAL', 'string'),
 ('LBL_GEO_TRAIN', 'string'),
 ('LBL_GRP_SEGMENT_NL', 'string'),
 ('LBL_SEGMENT_ANTICIPATION', 'string'),
 ('FLG_CMD_CARTE_1225', 'string'),
 ('anciennete', 'double'),
 ('recence_cmd', 'double'),
 ('AGE', 'double')]

quelles sont les features avec valeurs manquantes
remplacer les valeurs manquantes par -1 pour toutes les features

In [14]:
dm = [df.select(f.mean(feature)).collect()[0][0] for feature in var_quanti]
dm

[1.515022865927351,
 1.2363232480537965,
 152.64514973822276,
 43.19983674929222,
 0.3599612090605884,
 0.20166499766517768,
 0.3525474532671343,
 0.321143304334669,
 0.13100876267904052,
 0.6506145379939793,
 16.574795970884495,
 0.34099569046665423,
 1707.0163204391963,
 28.963997587586228,
 36.77269796022761]

In [15]:
dfs = df.select([f.when(df[feature].isNotNull(),df[feature])\
           .otherwise(-1).alias(feature) for i,feature in enumerate(client_cols_to_keep)]\
          +[f.when(df[feature].isNotNull(),df[feature])\
            .otherwise(dm[i]).alias(feature) for i,feature in enumerate(var_quanti)])

In [ ]:
dfs.toPandas()

In [16]:
def replace_missing_values(df, continuous, categorical) :
    dm = [df.select(f.mean(feature)).collect()[0][0] for feature in continuous]
    dfs = df.select([f.when(df[feature].isNotNull(),df[feature])\
                     .otherwise(-1).alias(feature) for i,feature in enumerate(categorical)]\
                    +[f.when(df[feature].isNotNull(),df[feature])\
                    .otherwise(dm[i]).alias(feature) for i,feature in enumerate(continuous)])
    return dfs
#dfvarna.select("LBL_STATUT_CLT").distinct().collect()`

In [17]:
dffin = replace_missing_values(df, var_quanti, var_quali)
dffin.toPandas()

,ID_CLIENT,LBL_STATUT_CLT,LBL_GEO_AIR,LBL_SEG_COMPORTEMENTAL,LBL_GEO_TRAIN,LBL_GRP_SEGMENT_NL,LBL_SEGMENT_ANTICIPATION,FLG_CMD_CARTE_1225,flg_cmd_lowcost,nb_od,...,mean_classe_1,mean_pointe,mean_depart_we,flg_track_nl_lowcost,flg_track_nl,days_since_last_visit,tx_conversion,anciennete,recence_cmd,AGE
0,000843db32fbaecfbb047ca0bb04b1f9f4d9425a,Grand,Aéroports de Paris Orly,Chasseurs Bons Plans,Paris,Spectateur,Mixte,0,-1.0,1.000000,...,0.000000,0.000000,0.000000,0.131009,0.650615,8.000000,0.111111,1550.0,36.0,36.772698
1,001338752ea32d9de129c8f8bdf3e2224cf0bd71,Grand,Aéroport de Marseille Provence (MRS),Comportement Pro,Marseille,Spectateur,Anticipateur,0,-1.0,1.000000,...,1.000000,0.000000,0.000000,0.131009,0.650615,3.000000,0.130435,1667.0,25.0,35.000000
2,003fb9dca8de374386d0fa97b570950583111931,Moyen moins,Aéroport de Lyon - Saint Exupéry,Rythmes scolaires,Lyon,Spectateur,Peu Anticipateur,1,1.0,3.000000,...,0.000000,0.250000,0.250000,0.000000,1.000000,15.000000,1.000000,395.0,15.0,25.000000
3,004efa6652e570ef68944b780738e159fbf2aeb5,Moyen moins,Aéroport de Lyon - Saint Exupéry,Sans contraintes,Lyon,Spectateur,Tres Anticipateur,0,-1.0,1.000000,...,0.500000,0.000000,0.500000,0.131009,0.650615,17.000000,0.125000,2188.0,20.0,31.000000
4,005dd0b718a8f4598ae3044e60f9c20222eb3a35,Petit,Aéroport de Strasbourg,Sans contraintes,Metz,Endormi,Non Anticipateur,0,-1.0,1.000000,...,0.000000,0.500000,0.500000,0.131009,0.650615,15.000000,0.333333,3005.0,15.0,32.000000
5,00631224500d2fa4d34b7ee149ca6460896e89fd,Grand,Aéroport de Rennes,Comportement Pro,Rennes,Spectateur,Non Anticipateur,1,-1.0,2.000000,...,0.666667,0.333333,0.666667,0.131009,0.650615,4.000000,0.034483,2094.0,6.0,25.000000
6,00660b90838bcbcc223018a3b0ccedefa94c34a1,Nouveau actif,-1,Nouveaux,-1,Non defini,Non Defini,0,-1.0,1.000000,...,0.000000,0.000000,0.000000,0.131009,0.650615,29.000000,0.111111,153.0,31.0,21.000000
7,008ad1b794a7ec0949d2e010830f2650903d69aa,Moyen moins,Aéroport Lille Lesquin,Sans contraintes,Lille,Non defini,Mixte,1,-1.0,2.000000,...,0.000000,0.000000,0.000000,0.131009,0.650615,12.000000,0.041667,1329.0,33.0,25.000000
8,008c6ed734788c21ccf402183fa982499a32ce68,Grand,Aéroports de Paris Roissy-Charles-de Gaulle,Sans contraintes,Paris,Non defini,Anticipateur,0,-1.0,1.000000,...,0.000000,0.307692,0.846154,0.131009,0.650615,2.000000,0.280000,1236.0,3.0,20.000000
9,0096985375e689bc3daf50ae50f359ecbd3c88e9,Petit,Aéroport de Toulouse Blagnac,Anticipateurs Methodiques,Toulouse,Acteur,Tres Anticipateur,0,-1.0,1.000000,...,1.000000,1.000000,0.000000,0.131009,0.650615,13.000000,0.500000,3591.0,13.0,36.772698


In [18]:
def input_df(df):
    ds = df.select('ID_CLIENT',
    f.when(df.LBL_GEO_TRAIN.isin(['Toulouse', 'Lille', 'Dijon',
                                  'Lyon', 'Marseille', 'Paris',
                                  'Nice', 'Limoges','Rouen','Rennes',
                                  'Montpellier', 'Bordeaux', 'Metz',
                                  'Strasbourg']), df.LBL_GEO_TRAIN)\
               .otherwise('na').alias('geo_train'),
    f.when(df.LBL_GEO_AIR.isin(['Aéroports de Paris Orly',
                                'Aéroport de Bâle-Mulhouse / Bassel',
                                'Aéroport Lille Lesquin', 'Aéroport de Rennes',
                                'Aéroport de Nantes Atlantique',
                                'Aéroport de Marseille Provence  (MRS)', 
                                'Aéroport de Bordeaux Mérignac',
                                'Aéroports de Paris Roissy-Charles-de Gaulle', 
                                "Aéroport de Nice Côte d'Azur",
                                'Aéroport de Strasbourg',
                                'Aéroport de Lyon - Saint Exupéry', 
                                'Aéroport de Toulouse Blagnac']), df.LBL_GEO_AIR)\
               .otherwise('na').alias('geo_air'),
    f.when(df.FLG_CMD_CARTE_1225 == '1', '1')\
                   .otherwise('0').alias('cc_jeunes'),
    f.when(df.LBL_STATUT_CLT.isin(['Tres grand', 'Nouveau actif',
                                   'Moyen moins', ' Prospect', ' Petit',
                                   'Inactif', 'Tres petit',
                                   'Nouveau prospect', 'Moyen plus',
                                   'Grand']), df.LBL_STATUT_CLT)\
                   .otherwise('na').alias('segt_rfm'),
    f.when(df.LBL_SEGMENT_ANTICIPATION.isin(['Peu Anticipateur', 'Tres Anticipateur',
                                             'Anticipateur', 'Mixte', 'Non Anticipateur',
                                             'Non Defini']), df.LBL_SEGMENT_ANTICIPATION)\
                   .otherwise('na').alias('segt_anticipation'),
    f.when(df.LBL_SEG_COMPORTEMENTAL.isin(['Mono-commande',
                                           'Comportement Pro',
                                           'Exclusifs Agence', 
                                           'Anticipateurs Methodiques',
                                           'Chasseurs Bons Plans', 
                                           'Rythmes scolaires', 'Nouveaux',
                                           'Sans contraintes']),
           df.LBL_SEG_COMPORTEMENTAL).otherwise('na').alias('segt_comportemental'), 
    f.when(df.LBL_GRP_SEGMENT_NL.isin(['Endormi', 'Spectateur', 'Acteur',
                                       'Eteint', 'Non defini']),
           df.LBL_GRP_SEGMENT_NL).otherwise('na').alias('segt_nl'),
    f.when(((df.AGE > 0) & (df.AGE < 100)), df.AGE)\
                   .otherwise(-1).alias('age'),
    f.when(df.recence_cmd >= 0, df.recence_cmd)\
                   .otherwise(-1).alias('recence_cmd'),
    f.when(((df.mean_duree_voyage > 0) & (df.mean_duree_voyage < 750)),
           df.mean_duree_voyage).otherwise(-1).alias('mean_duree_voyage'),
    f.when(df.days_since_last_visit >= 0, df.days_since_last_visit)\
                   .otherwise(-1).alias('recence_visite'),
    f.when(df.mean_mt_voyage > 0, df.mean_mt_voyage)\
                   .otherwise(-1).alias('mean_mt_voyage'),
    f.when(df.anciennete >= 0, df.anciennete)\
                   .otherwise(-1).alias('anciennete'),
    f.when(df.nb_od > 0, df.nb_od)\
                   .otherwise(-1).alias('nb_od'),
    f.when(df.mean_nb_passagers > 0, df.mean_nb_passagers)\
                   .otherwise(-1).alias('mean_nb_passagers'),
    f.when(df.mean_tarif_loisir >= 0, df.mean_tarif_loisir)\
                   .otherwise(-1).alias('mean_tarif_loisir'),
    f.when(df.mean_classe_1 >= 0, df.mean_classe_1)\
                   .otherwise(-1).alias('mean_classe_1'),
    f.when(df.mean_pointe >= 0, df.mean_pointe)\
                   .otherwise(-1).alias('mean_pointe'),
    f.when(df.mean_depart_we >= 0, df.mean_depart_we)\
                   .otherwise(-1).alias('mean_depart_we'),
    f.when(df.tx_conversion >= 0, df.tx_conversion)\
                   .otherwise(-1).alias('tx_conversion'),
    f.when(df.flg_cmd_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_cmd_lowcost'),
    f.when(df.flg_track_nl_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_track_nl_lowcost'), 
    f.when(df.flg_track_nl == 1, '1')\
                   .otherwise('0').alias('flg_track_nl'))
    
    return ds


In [19]:
dfin = input_df(dffin)

Quelles sont les differentes valeurs de notre label : flg_cmd_lowcost

In [20]:
dfin.groupby("flg_cmd_lowcost").agg(f.count("flg_cmd_lowcost").alias("nbre")).show()

+---------------+------+
|flg_cmd_lowcost|  nbre|
+---------------+------+
|              0|979911|
|              1|104306|
+---------------+------+



In [21]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression


In [22]:
y=dfin.select("flg_cmd_lowcost").collect()

In [ ]:
y

##### features engineering et modélisation

In [23]:

def preprocessed_df(df, label="flg_cmd_lowcostIndex"):
    max_values_to_define_str_cols = 10
    id_col = 'ID_CLIENT'
    
    dty = dict(df.dtypes)
    str_cols = [k for k, v in dty.items() if v == 'string']
    str_cols.remove(id_col)
    
    for c in str_cols:
        stringIndexer = StringIndexer(inputCol=c, outputCol=c+"Index")
        model_str = stringIndexer.fit(df)
        df = model_str.transform(df).drop(c)

    input_cols = df.columns
    input_cols.remove(id_col)
    input_cols.remove(label)
    
    assembler = VectorAssembler(inputCols=input_cols,
                            outputCol="features")
    df = assembler.transform(df)
    
    featureIndexer = VectorIndexer(inputCol="features", 
                   outputCol="indexedFeatures", 
                   maxCategories=max_values_to_define_str_cols).fit(df)
    return featureIndexer.transform(df), df


data, dff = preprocessed_df(dfin)



prelever un sample de data pour notre modelisation

In [50]:
#donnees d'apprentissage
df_train = data.sample(False, 0.5, 42)
df_val = data.subtract(df_train)

In [51]:
df_train.toPandas().head()

,ID_CLIENT,age,recence_cmd,mean_duree_voyage,recence_visite,mean_mt_voyage,anciennete,nb_od,mean_nb_passagers,mean_tarif_loisir,...,cc_jeunesIndex,segt_rfmIndex,segt_anticipationIndex,segt_comportementalIndex,segt_nlIndex,flg_cmd_lowcostIndex,flg_track_nl_lowcostIndex,flg_track_nlIndex,features,indexedFeatures
0,005dd0b718a8f4598ae3044e60f9c20222eb3a35,32.0,15.0,84.000000,15.0,65.5,3005.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,"[32.0, 15.0, 84.0, 15.0, 65.5, 3005.0, 1.0, 1....","[32.0, 15.0, 84.0, 15.0, 65.5, 3005.0, 1.0, 1...."
1,00660b90838bcbcc223018a3b0ccedefa94c34a1,21.0,31.0,67.000000,29.0,8.0,153.0,1.0,1.0,0.0,...,0.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,"[21.0, 31.0, 67.0, 29.0, 8.0, 153.0, 1.0, 1.0,...","[21.0, 31.0, 67.0, 29.0, 8.0, 153.0, 1.0, 1.0,..."
2,008ad1b794a7ec0949d2e010830f2650903d69aa,25.0,33.0,146.500000,12.0,31.1,1329.0,2.0,1.5,0.0,...,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,"[25.0, 33.0, 146.5, 12.0, 31.1, 1329.0, 2.0, 1...","[25.0, 33.0, 146.5, 12.0, 31.1, 1329.0, 2.0, 1..."
3,008c6ed734788c21ccf402183fa982499a32ce68,20.0,3.0,180.384615,2.0,-1.0,1236.0,1.0,1.0,0.0,...,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,"(20.0, 3.0, 180.3846153846154, 2.0, -1.0, 1236...","(20.0, 3.0, 180.3846153846154, 2.0, -1.0, 1236..."
4,00b2a46ecd1c941fabdd8074ba29af412d581348,61.0,22.0,234.750000,3.0,74.0,397.0,3.0,1.0,0.0,...,0.0,5.0,0.0,5.0,4.0,0.0,0.0,0.0,"[61.0, 22.0, 234.75, 3.0, 74.0, 397.0, 3.0, 1....","[61.0, 22.0, 234.75, 3.0, 74.0, 397.0, 3.0, 1...."


In [ ]:
df_train.head()

#### Logistic regression

quelle est le label est renseigne pour la modelisation?

In [61]:
##Construction de modele regression_logistique
lr = LogisticRegression(labelCol="flg_cmd_lowcostIndex",
                        featuresCol="indexedFeatures",elasticNetParam=0.5)

lr_model = lr.fit(df_train)

#reg_log = lr_model.evaluate(df_train)

#reg_log.areaUnderROC

In [62]:
reg_log = lr_model.evaluate(df_val)

In [63]:
reg_log.areaUnderROC

0.9071396196941249

In [45]:
reg_log1 = lr_model.transform(df_train)

In [47]:
reg_log1 = lr_model.evaluate(df_val)

In [49]:
reg_log1.areaUnderROC

0.9045117023043822

In [64]:
reg_log.accuracy

0.9659999151105132

In [57]:
reg_log.predictions

DataFrame[ID_CLIENT: string, age: double, recence_cmd: double, mean_duree_voyage: double, recence_visite: double, mean_mt_voyage: double, anciennete: double, nb_od: double, mean_nb_passagers: double, mean_tarif_loisir: double, mean_classe_1: double, mean_pointe: double, mean_depart_we: double, tx_conversion: double, geo_trainIndex: double, geo_airIndex: double, cc_jeunesIndex: double, segt_rfmIndex: double, segt_anticipationIndex: double, segt_comportementalIndex: double, segt_nlIndex: double, flg_cmd_lowcostIndex: double, flg_track_nl_lowcostIndex: double, flg_track_nlIndex: double, features: vector, indexedFeatures: vector, rawPrediction: vector, probability: vector, prediction: double]

In [60]:
reg_log.roc

DataFrame[FPR: double, TPR: double]

Ajuster le modele de regression logistique et calculer les coefficients de notre modele.

Evaluer votre modele courbe roc, precision, rappel, etc....

predire alors les clients lowcoast sur un sample de data n'ayant pas servi à l'apprentissage

##### Random Forest

In [ ]:
classifier = RandomForestClassifier(labelCol="flg_cmd_lowcostIndex", 
                                    featuresCol="indexedFeatures",
                                    maxDepth=15, numTrees=100)

model_rf = classifier.fit(data1)

evaluer les performance de notre modele

In [ ]:
# une autre technique pour evaluer le modele

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

Evaluer soit meme le score en calculant le nombre de VP, FP, VN et FN
on calculera alors le score qui VP+VN/VP+VN+FP+FN
nb: la prediction est automatiquement creee dans le data set et correspond à la colonne prediction

In [ ]:
calculer egalement le rappel et la precision

In [ ]:
from pyspark.ml.linalg import Vectors
scoreAndLabels = map(lambda x: (Vectors.dense([1.0 - x[0], x[0]]), x[1]),
                     [(0.1, 0.0), (0.1, 1.0), (0.4, 0.0), (0.6, 0.0), (0.6, 1.0), (0.6, 1.0), (0.8, 1.0)])
dataset = spark_session.createDataFrame(scoreAndLabels, ["raw", "label"])
dataset.head()

#### how to create a spark data frame

In [ ]:
from pyspark.sql import Row
import datetime

l = [(datetime.date(2018,1,3), 'Ankit',25, "Female"),
     (datetime.date(2018,2,3), 'Jalfaizy',22, "Female"),
     (datetime.date(2018,1,5), 'saurabh',20, "Female"),
     (datetime.date(2018,1,12), 'Bala',26, "Female"),
     (datetime.date(2018,7,9), 'Jules',19, "Female") ,
     (datetime.date(2018,3,18), 'Arild',43, "Male"),
     (datetime.date(2018,1,5), 'sarah',20, "Male"),
     (datetime.date(2018,8,12), 'Boly',33, "Male"),
     (datetime.date(2018,4,6), 'Anita',35, "Male"),
     (datetime.date(2018,12,6), 'Jules',22, "Male"),
     (datetime.date(2018,7,24), 'Soul',20, "Female"),
     (datetime.date(2018,6,17), 'Gral',54, "Female"),
     (datetime.date(2018,9,7), 'Apoh',18, "Female"),
     (datetime.date(2018,10,4), 'Dony',32, "Female"),
     (datetime.date(2018,2,5), 'Tanoh',31, "Female"),
     (datetime.date(2018,11,12), 'Issouf',27, "Male"),
     (datetime.date(2018,10,3), 'Bilé',29, "Female"),
     (datetime.date(2018,5,3), 'Gagnon',20, "Female"),
     (datetime.date(2018,3,5), 'Papiss',28, "Female"),
     (datetime.date(2018,2,12), 'Kravitz',34, "Female"),
     (datetime.date(2018,5,9), 'Mouli',35, "Female"),
     (datetime.date(2018,8,3), 'Jacques',27, "Male"),
     (datetime.date(2018,12,5), 'soum',22, "Male"),
     (datetime.date(2018,4,12), 'MBra',36, "Male")]

rdd = spark_session.sparkContext.parallelize(l)
people = rdd.map(lambda x: Row(date=x[0], name=x[1], age=int(x[2]), sexe = x[3]))
schemaPeople = spark_session.createDataFrame(people)
schemaPeople.toPandas()

1- compter le nombre de personne totale 
2- compter le nombre de fille et de garcon
3- quel est l'age moyen, median mini et maxi dans chaque groupe (garcon, fille)
4 - quelle est la date de dernière visite de chaque client par rapport à la date d'aujourd'hui (la colonne date correspond à la date de visite)
5 - quels sont les personnes qui ont une date de visite < 400 jours.

### 1- compter le nombre de personne totale 

In [ ]:
schemaPeople.count()

###  2- compter le nombre de fille et de garcon 

In [ ]:
schemaPeople.groupby("sexe").agg(f.count("sexe").alias("nb_pers")).show()

### 3- quel est l'age moyen, median mini et maxi dans chaque groupe (garcon, fille)

In [ ]:
schemaPeople.groupby("sexe").agg(f.mean("age").alias("moyenne"),
                                 f.max("age").alias("max"),
                                 f.min("age").alias("min"),
                                 f.count("age").alias("nbre_pers")).show()

### 4 - quelle est la date de dernière visite de chaque client par rapport à la date d'aujourd'hui (la colonne date correspond à la date de visite)

In [ ]:
jour = datetime.datetime.today().strftime('%Y-%m-%d')
dd = schemaPeople.select("*", f.lit(jour).alias("date_max"))

dd.select("*", f.datediff('date_max', 'date').alias('days_since_last_visit'))

In [ ]:
df = dd.select("*", (f.datediff('date_max', 'date')<400).alias('<400?'))


In [ ]:
df.groupby("<400?").agg(f.count("<400?").alias("nb_visite<400")).show()

In [ ]:
schemaPeople.filter(schemaPeople["name"].isNull()).show()

Creer une colonne adulte qui prendra en compte 

In [ ]:
df2 = schemaPeople.select("*", f.month('date').alias('mois'))
df2.toPandas()

In [ ]:
schemaPeople.withColumn("adulte", f.when(schemaPeople["age"] < 25, 0).when(schemaPeople["age"] >= 25,1)).toPandas()

In [ ]:
dd = schemaPeople.select("*",
                         f.lit(datetime.date.today()).alias("date_max"))
dd.select("*", f.datediff('date_max', 'date')\
                    .alias('days_since_last_visit')).toPandas()